<a href="https://colab.research.google.com/github/pdsw11/test/blob/main/pitch_did2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd, re, unicodedata

# === 0) 読み込み ===
sv = pd.read_csv("savant_2022.csv")
sv["year"] = 2022  # 念のため年度列を付ける

# === 1) 名前の整形 ===
# "Burnes, Corbin" → "Corbin Burnes"
def swap_name(s: str) -> str:
    if isinstance(s, str) and "," in s:
        last, first = s.split(",", 1)
        return f"{first.strip()} {last.strip()}"
    return s

# アクセント除去・小文字化・記号除去
def norm_name(s: str) -> str:
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace(",", " ").replace(".", " ").replace("*", " ")
    s = re.sub(r"\b(jr|sr|ii|iii|iv|v)\b", " ", s)
    s = re.sub(r"[^a-z\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

sv["player_name_clean"] = sv["player_name"].map(swap_name)
sv["name_norm"] = sv["player_name_clean"].map(norm_name)

In [ ]:
# 研究で使える変数リスト（球速・回転・フォーム・打球質など）
use_cols = [
    "player_name", "player_id",
    "velocity", "effective_speed", "spin_rate", "release_extension", "arm_angle",
    "launch_speed", "launch_angle", "hardhit_percent", "swing_miss_percent",
    "total_pitches", "pa"
]

# ファイルに存在する列だけ残す
available = [c for c in use_cols if c in sv.columns]
keep = ["name_norm", "year", "player_name_clean"] + available

savant22_clean = (
    sv[keep]
    .drop_duplicates(subset=["name_norm", "year"])
    .reset_index(drop=True)
)

In [ ]:
print("savant22_clean shape:", savant22_clean.shape)
print(savant22_clean.head(10))

# 必要なら保存
savant22_clean.to_csv("savant22_clean.csv", index=False)
print("✅ Saved: savant22_clean.csv")

savant22_clean shape: (869, 16)
              name_norm  year     player_name_clean            player_name  \
0         jose alvarado  2022         José Alvarado         Alvarado, José   
1      brusdar graterol  2022      Brusdar Graterol      Graterol, Brusdar   
2        emmanuel clase  2022        Emmanuel Clase        Clase, Emmanuel   
3          gregory soto  2022          Gregory Soto          Soto, Gregory   
4         hunter harvey  2022         Hunter Harvey         Harvey, Hunter   
5           jhoan duran  2022           Jhoan Duran           Duran, Jhoan   
6  seranthony dominguez  2022  Seranthony Domínguez  Domínguez, Seranthony   
7         kyle finnegan  2022         Kyle Finnegan         Finnegan, Kyle   
8          jacob degrom  2022          Jacob deGrom          deGrom, Jacob   
9            nick duron  2022            Nick Duron            Duron, Nick   

   player_id  velocity  effective_speed  spin_rate  release_extension  \
0     621237      96.9            97

In [ ]:
import re, unicodedata

def norm_name(s: str) -> str:
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace(",", " ").replace(".", " ").replace("*", " ")
    s = re.sub(r"\b(jr|sr|ii|iii|iv|v)\b", " ", s)
    s = re.sub(r"[^a-z\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [ ]:
# すでに作成済み: savant22_clean  （name_norm / year / 各指標）
# ここで「pre接尾辞」に統一しておく（存在する列だけリネーム）
pre_map = {
    "velocity": "avg_velo_pre",
    "effective_speed": "effective_speed_pre",
    "spin_rate": "spin_rate_pre",
    "release_extension": "release_ext_pre",
    "arm_angle": "arm_angle_pre",
    "launch_speed": "launch_speed_pre",
    "launch_angle": "launch_angle_pre",
    "hardhit_percent": "hardhit_percent_pre",
    "swing_miss_percent": "swing_miss_percent_pre",
    "total_pitches": "total_pitches_2022",
    "pa": "pa_2022",
}
sv_cols = ["name_norm", "year"] + [c for c in pre_map if c in savant22_clean.columns]
savant22_pre = savant22_clean[sv_cols].rename(columns={k:v for k,v in pre_map.items() if k in savant22_clean.columns})

In [ ]:
import pandas as pd, re, unicodedata

# --- 成績データの読み込み ---
pitching_2022 = pd.read_csv("pitching_2022.csv")

# --- 名前標準化関数（前と同じ） ---
def norm_name(s: str) -> str:
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace(",", " ").replace(".", " ").replace("*", " ")
    s = re.sub(r"\b(jr|sr|ii|iii|iv|v)\b", " ", s)
    s = re.sub(r"[^a-z\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# --- 投手名の列を探す（例: Player or Name）---
name_col = next((c for c in ["Player","player_name","Name"] if c in pitching_2022.columns), None)
if name_col is None:
    raise ValueError("投手名の列が見つかりません。'Player' などを確認してください。")

# --- 名前の正規化＆year列の追加 ---
pitching_2022["name_norm"] = pitching_2022[name_col].map(norm_name)
pitching_2022["year"] = 2022

# --- マージ ---
merged_2022 = pitching_2022.merge(savant22_clean, on=["name_norm","year"], how="left")

# --- 確認 ---
print("merged_2022 shape:", merged_2022.shape)
print(merged_2022[[name_col, "velocity", "effective_speed", "spin_rate", "release_extension", "arm_angle"]].head())

merged_2022 shape: (1069, 54)
            Player  velocity  effective_speed  spin_rate  release_extension  \
0  Sandy Alcantara      94.4            93.58     2218.0               5.90   
1       Aaron Nola      87.5            88.17     2234.0               6.89   
2    Miles Mikolas      87.8            87.92     2327.0               6.42   
3    Corbin Burnes      91.6            91.92     2601.0               6.38   
4  Framber Valdez*      88.2            87.30     2380.0               5.91   

   arm_angle  
0       33.9  
1       20.9  
2       47.9  
3       41.3  
4       43.0  


In [ ]:
vsc22=pitching_2022.copy()

In [ ]:
import numpy as np

# 1) 列名をpreに（存在する列だけ）
rename_map = {
    "velocity": "avg_velo_pre",
    "effective_speed": "effective_speed_pre",
    "spin_rate": "spin_rate_pre",
    "release_extension": "release_ext_pre",
    "arm_angle": "arm_angle_pre",
}
merged_2022 = merged_2022.rename(columns={k:v for k,v in rename_map.items() if k in merged_2022.columns})

# 2) 診断
pre_cols = [c for c in ["avg_velo_pre","effective_speed_pre","spin_rate_pre",
                        "release_ext_pre","arm_angle_pre"] if c in merged_2022.columns]
print("欠損率:\n", merged_2022[pre_cols].isna().mean().sort_values(ascending=False))

# ちょい重複が無いか（name_norm×year が1行か）
if "name_norm" in merged_2022.columns and "year" in merged_2022.columns:
    dup_rate = merged_2022.duplicated(subset=["name_norm","year"]).mean()
    print("name_norm×year 重複率:", dup_rate)

# 3) km/h（任意）
if "avg_velo_pre" in merged_2022.columns:
    merged_2022["avg_velo_pre_kmh"] = merged_2022["avg_velo_pre"] * 1.60934

# 4) 保存
merged_2022.to_csv("pitching_vsc_2022_with_savant.csv", index=False)
print("✅ Saved: pitching_vsc_2022_with_savant.csv")

欠損率:
 avg_velo_pre           0.004677
effective_speed_pre    0.004677
spin_rate_pre          0.004677
release_ext_pre        0.004677
arm_angle_pre          0.004677
dtype: float64
name_norm×year 重複率: 0.1880261927034612
✅ Saved: pitching_vsc_2022_with_savant.csv


In [ ]:
print(merged_2022[["avg_velo_pre","effective_speed_pre"]].corr(numeric_only=True))
print(merged_2022[["spin_rate_pre","hardhit_percent"]].corr(numeric_only=True) if "hardhit_percent" in merged_2022.columns else "hardhit_percentなし")

                     avg_velo_pre  effective_speed_pre
avg_velo_pre             1.000000             0.996155
effective_speed_pre      0.996155             1.000000
                 spin_rate_pre  hardhit_percent
spin_rate_pre         1.000000         0.029489
hardhit_percent       0.029489         1.000000


In [ ]:
#重複消去
merged_2022 = merged_2022.sort_values("IP", ascending=False).drop_duplicates(subset=["name_norm","year"], keep="first")
print("After dedup:", merged_2022.shape)

After dedup: (868, 55)


In [ ]:
import pandas as pd, re, unicodedata

# === 1) 読み込み（エンコ自動切替版を使うなら load_csv を利用） ===
def load_csv(path):
    try:
        return pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin1")

tempo22 = load_csv("pitch_tempo2022.csv")

# === 2) 名前整形: "姓, 名" → "名 姓" & 正規化 ===
def swap_name(s: str) -> str:
    if isinstance(s, str) and "," in s:
        last, first = s.split(",", 1)
        return f"{first.strip()} {last.strip()}"
    return s

def norm_name(s: str) -> str:
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace(",", " ").replace(".", " ").replace("*", " ")
    s = re.sub(r"\b(jr|sr|ii|iii|iv|v)\b", " ", s)
    s = re.sub(r"[^a-z\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# tempo22 = tempo22.copy() # この行は不要なので削除
tempo22["player_name_clean"] = tempo22["entity_name"].map(swap_name)
tempo22["name_norm"] = tempo22["player_name_clean"].map(norm_name)
tempo22["year"] = 2022

# === 3) テンポ列の決定（.1 があれば優先度を決めて1本に統一） ===
tempo_cols = [c for c in ["median_seconds_empty", "median_seconds_empty.1"] if c in tempo22.columns]
if not tempo_cols:
    raise ValueError("テンポ列（median_seconds_empty）が見つかりません。")
# 基本は元の列を優先、欠損だけ .1 で補完
tempo22["tempo"] = tempo22[tempo_cols[0]]
if len(tempo_cols) > 1:
    tempo22["tempo"] = tempo22["tempo"].fillna(tempo22[tempo_cols[1]])

# 1投手1行（name_norm×year で重複除去。複数行がある場合は投球数多い方を採用）
if "total_pitches_empty" in tempo22.columns:
    tempo22 = tempo22.sort_values("total_pitches_empty", ascending=False)
tempo_use = (tempo22[["name_norm","year","tempo"]]
             .dropna(subset=["name_norm"])
             .drop_duplicates(subset=["name_norm","year"]))

print("tempo_use shape:", tempo_use.shape)
print(tempo_use.head())

# === 4) 成績+Savant（さっきの merged_2022）へマージ ===
# merged_2022 がメモリにある前提
merged_2022 = merged_2022.merge(tempo_use, on=["name_norm","year"], how="left")\
                         .rename(columns={"tempo":"tempo_pre"})

print("merged_2022 shape:", merged_2022.shape)
print("tempo_pre missing rate:", merged_2022["tempo_pre"].isna().mean())
print(merged_2022[["Player","avg_velo_pre","spin_rate_pre","tempo_pre"]].head(10))

# === 5) 保存（任意） ===
merged_2022.to_csv("pitching_vsc_2022_with_savant_tempo.csv", index=False)
print("✅ Saved: pitching_vsc_2022_with_savant_tempo.csv")

tempo_use shape: (390, 3)
           name_norm  year   tempo
334    corbin burnes  2022  21.088
170      gerrit cole  2022  17.599
19     miles mikolas  2022  14.902
110  adam wainwright  2022  16.797
118       zac gallen  2022  16.914
merged_2022 shape: (868, 56)
tempo_pre missing rate: 0.5506912442396313
            Player  avg_velo_pre  spin_rate_pre  tempo_pre
0  Sandy Alcantara          94.4         2218.0     15.462
1       Aaron Nola          87.5         2234.0     20.119
2    Miles Mikolas          87.8         2327.0     14.902
3    Corbin Burnes          91.6         2601.0     21.088
4  Framber Valdez*          88.2         2380.0     17.758
5      Gerrit Cole          93.1         2433.0     17.599
6    Merrill Kelly          90.0         2311.0     16.974
7     Shane Bieber          87.0         2326.0     14.550
8      Alek Manoah          89.5         2238.0     20.700
9    Martín Pérez*          89.1         1986.0     18.191
✅ Saved: pitching_vsc_2022_with_savant_temp

In [ ]:
import pandas as pd, re, unicodedata

# === 読み込み ===
pitching_2023 = pd.read_csv("pitching_2023.csv")
pitching_2023["year"] = 2023

# === 名前標準化 ===
def norm_name(s: str) -> str:
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace(",", " ").replace(".", " ").replace("*", " ")
    s = re.sub(r"\b(jr|sr|ii|iii|iv|v)\b", " ", s)
    s = re.sub(r"[^a-z\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

name_col = next((c for c in ["Player","player_name","Name"] if c in pitching_2023.columns), None)
pitching_2023["name_norm"] = pitching_2023[name_col].map(norm_name)

In [ ]:
# === 読み込み ===
sv23 = pd.read_csv("savant_2023.csv")
sv23["year"] = 2023

# 名前整形
def swap_name(s: str) -> str:
    if isinstance(s, str) and "," in s:
        last, first = s.split(",", 1)
        return f"{first.strip()} {last.strip()}"
    return s

sv23["player_name_clean"] = sv23["player_name"].map(swap_name)
sv23["name_norm"] = sv23["player_name_clean"].map(norm_name)

# 必要列の抽出
use_cols = [
    "player_name", "player_id",
    "velocity", "effective_speed", "spin_rate", "release_extension", "arm_angle",
    "launch_speed", "launch_angle", "hardhit_percent", "swing_miss_percent",
    "total_pitches", "pa"
]
available = [c for c in use_cols if c in sv23.columns]
keep = ["name_norm", "year", "player_name_clean"] + available
savant23_clean = (
    sv23[keep]
      .drop_duplicates(subset=["name_norm", "year"])
      .reset_index(drop=True)
)

In [ ]:
# === 読み込み ===
sv23 = pd.read_csv("savant_2023.csv")
sv23["year"] = 2023

# 名前整形
def swap_name(s: str) -> str:
    if isinstance(s, str) and "," in s:
        last, first = s.split(",", 1)
        return f"{first.strip()} {last.strip()}"
    return s

sv23["player_name_clean"] = sv23["player_name"].map(swap_name)
sv23["name_norm"] = sv23["player_name_clean"].map(norm_name)

# 必要列の抽出
use_cols = [
    "player_name", "player_id",
    "velocity", "effective_speed", "spin_rate", "release_extension", "arm_angle",
    "launch_speed", "launch_angle", "hardhit_percent", "swing_miss_percent",
    "total_pitches", "pa"
]
available = [c for c in use_cols if c in sv23.columns]
keep = ["name_norm", "year", "player_name_clean"] + available
savant23_clean = (
    sv23[keep]
      .drop_duplicates(subset=["name_norm", "year"])
      .reset_index(drop=True)
)

In [ ]:
import pandas as pd, re, unicodedata

def norm_name(s: str) -> str:
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace(",", " ").replace(".", " ").replace("*", " ")
    s = re.sub(r"\b(jr|sr|ii|iii|iv|v)\b", " ", s)
    s = re.sub(r"[^a-z\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# === 1) 成績データ ===
pitch23 = pd.read_csv("pitching_2023.csv")
pitch23["name_norm"] = pitch23[pitch23.columns[0]].map(norm_name)  # 仮: 1列目がPlayer列ならOK
print("🎯 Pitching_2023")
print("総行数:", len(pitch23))
print("ユニーク投手数:", pitch23["name_norm"].nunique())
print("重複率:", 1 - pitch23["name_norm"].nunique() / len(pitch23))
print()

# === 2) Savantデータ ===
sv23 = pd.read_csv("savant_2023.csv")
sv23["player_name_clean"] = sv23["player_name"].map(lambda s: s.split(",")[1].strip() + " " + s.split(",")[0].strip() if isinstance(s,str) and "," in s else s)
sv23["name_norm"] = sv23["player_name_clean"].map(norm_name)
print("🎯 Savant_2023")
print("総行数:", len(sv23))
print("ユニーク投手数:", sv23["name_norm"].nunique())
print("重複率:", 1 - sv23["name_norm"].nunique() / len(sv23))
print()

# === 3) Pitch Tempoデータ ===
tempo23 = pd.read_csv("pitch_tempo2023.csv", encoding="latin1")
tempo23["player_name_clean"] = tempo23["entity_name"].map(lambda s: s.split(",")[1].strip() + " " + s.split(",")[0].strip() if isinstance(s,str) and "," in s else s)
tempo23["name_norm"] = tempo23["player_name_clean"].map(norm_name)
print("🎯 Tempo_2023")
print("総行数:", len(tempo23))
print("ユニーク投手数:", tempo23["name_norm"].nunique())
print("重複率:", 1 - tempo23["name_norm"].nunique() / len(tempo23))

🎯 Pitching_2023
総行数: 1065
ユニーク投手数: 2
重複率: 0.9981220657276996

🎯 Savant_2023
総行数: 863
ユニーク投手数: 862
重複率: 0.0011587485515642815

🎯 Tempo_2023
総行数: 386
ユニーク投手数: 386
重複率: 0.0


In [ ]:
merged_2023 = pitching_2023.merge(savant23_clean, on=["name_norm","year"], how="left")

# pre用にリネーム（2023はpost用にしてもOK）
rename_map = {
    "velocity": "avg_velo_post",
    "effective_speed": "effective_speed_post",
    "spin_rate": "spin_rate_post",
    "release_extension": "release_ext_post",
    "arm_angle": "arm_angle_post"
}
merged_2023 = merged_2023.rename(columns={k:v for k,v in rename_map.items() if k in merged_2023.columns})

# 重複除去（IPが多い方を優先）
if "IP" in merged_2023.columns:
    merged_2023 = merged_2023.sort_values("IP", ascending=False)\
                             .drop_duplicates(subset=["name_norm","year"], keep="first")

print("merged_2023 shape:", merged_2023.shape)
print(merged_2023[[name_col, "avg_velo_post", "spin_rate_post"]].head(10))

merged_2023 shape: (862, 54)
             Player  avg_velo_post  spin_rate_post
0        Logan Webb           88.6          1855.0
1        Zac Gallen           89.5          2235.0
2       Gerrit Cole           92.6          2432.0
3     Miles Mikolas           87.5          2279.0
4     Chris Bassitt           85.6          2134.0
5   Framber Valdez*           89.5          2319.0
6     Luis Castillo           92.6          2206.0
7      Mitch Keller           90.1          2454.0
8       Pablo López           89.7          2304.0
10       Aaron Nola           87.0          2283.0


In [ ]:
import pandas as pd, re, unicodedata

# === 読み込み（エンコーディング対策あり） ===
def load_csv(path):
    try:
        return pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin1")

tempo23 = load_csv("pitch_tempo2023.csv")

# === 名前整形（同じ関数を再利用） ===
def swap_name(s: str) -> str:
    if isinstance(s, str) and "," in s:
        last, first = s.split(",", 1)
        return f"{first.strip()} {last.strip()}"
    return s

def norm_name(s: str) -> str:
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.replace(",", " ").replace(".", " ").replace("*", " ")
    s = re.sub(r"\b(jr|sr|ii|iii|iv|v)\b", " ", s)
    s = re.sub(r"[^a-z\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

tempo23["player_name_clean"] = tempo23["entity_name"].map(swap_name)
tempo23["name_norm"] = tempo23["player_name_clean"].map(norm_name)
tempo23["year"] = 2023

# === テンポ列統合（.1の補完あり） ===
tempo_cols = [c for c in ["median_seconds_empty","median_seconds_empty.1"] if c in tempo23.columns]
tempo23["tempo"] = tempo23[tempo_cols[0]]
if len(tempo_cols) > 1:
    tempo23["tempo"] = tempo23["tempo"].fillna(tempo23[tempo_cols[1]])

# 投球数が多い方を優先（重複除去）
if "total_pitches_empty" in tempo23.columns:
    tempo23 = tempo23.sort_values("total_pitches_empty", ascending=False)
tempo_use23 = tempo23[["name_norm","year","tempo"]].drop_duplicates(subset=["name_norm","year"])

# === マージ ===
merged_2023 = merged_2023.merge(tempo_use23, on=["name_norm","year"], how="left")\
                         .rename(columns={"tempo":"tempo_post"})

# === 確認 ===
print("merged_2023 shape:", merged_2023.shape)
print("tempo_post missing rate:", merged_2023["tempo_post"].isna().mean())
print(merged_2023[["Player","avg_velo_post","spin_rate_post","tempo_post"]].head(10))

# 保存
merged_2023.to_csv("pitching_vsc_2023_with_savant_tempo.csv", index=False)
print("✅ Saved: pitching_vsc_2023_with_savant_tempo.csv")

merged_2023 shape: (862, 55)
tempo_post missing rate: 0.5545243619489559
            Player  avg_velo_post  spin_rate_post  tempo_post
0       Logan Webb           88.6          1855.0      13.131
1       Zac Gallen           89.5          2235.0      15.622
2      Gerrit Cole           92.6          2432.0      14.873
3    Miles Mikolas           87.5          2279.0      12.601
4    Chris Bassitt           85.6          2134.0      17.011
5  Framber Valdez*           89.5          2319.0      16.445
6    Luis Castillo           92.6          2206.0      12.930
7     Mitch Keller           90.1          2454.0      12.125
8      Pablo López           89.7          2304.0      15.972
9       Aaron Nola           87.0          2283.0      18.115
✅ Saved: pitching_vsc_2023_with_savant_tempo.csv


In [ ]:
import numpy as np

# 1) 2022で treated を作り直す
tempo_col_22 = "tempo_pre" if "tempo_pre" in merged_2022.columns else ("tempo" if "tempo" in merged_2022.columns else None)
if tempo_col_22 is None:
    raise KeyError("2022データに tempo_pre（または tempo）列が見つかりません。")

mask = merged_2022[tempo_col_22].notna()
# 外れ値を軽くクリップ（任意）
merged_2022.loc[mask, tempo_col_22] = merged_2022.loc[mask, tempo_col_22].clip(12, 25)

median_tempo = merged_2022.loc[mask, tempo_col_22].median()
print("Tempo median (2022):", median_tempo)

merged_2022["treated"] = np.where(
    merged_2022[tempo_col_22] > median_tempo, 1,
    np.where(merged_2022[tempo_col_22].isna(), np.nan, 0)
)
print("treated counts (2022):")
print(merged_2022["treated"].value_counts(dropna=False))

# 2) 2年分を結合して post と treated を付与
base_all = pd.concat([merged_2022, merged_2023], ignore_index=True)
base_all["post"] = (base_all["year"] == 2023).astype(int)

treated_map = merged_2022.set_index("name_norm")["treated"]
base_all["treated"] = base_all["name_norm"].map(treated_map)

# 3) 確認
print(base_all.groupby(["year","treated"]).size())
print("shape:", base_all.shape)

Tempo median (2022): 18.1095
treated counts (2022):
treated
NaN    478
0.0    195
1.0    195
Name: count, dtype: int64
year  treated
2022  0.0        195
      1.0        195
2023  0.0        172
      1.0        175
dtype: int64
shape: (1730, 64)


In [ ]:
base_all = pd.concat([merged_2022, merged_2023], ignore_index=True)

# POSTダミー
base_all["post"] = (base_all["year"] == 2023).astype(int)

# treated（tempo_pre の中央値で固定済み）
# → 2023にも同じ treated 値をコピー
treated_map = merged_2022.set_index("name_norm")["treated"]
base_all["treated"] = base_all["name_norm"].map(treated_map)

# データチェック
print(base_all.groupby(["year","treated"]).size())
print("shape:", base_all.shape)

year  treated
2022  0.0        195
      1.0        195
2023  0.0        172
      1.0        175
dtype: int64
shape: (1730, 64)


In [ ]:
import numpy as np
import pandas as pd

an_cols = ["BB9","ERA","Age","IP","avg_velo_pre","spin_rate_pre","treated","post","name_norm","year"]
use = base_all.copy()

# 解析に必要な列だけ（存在するものを使う）
have = [c for c in an_cols if c in use.columns]
use = use[have].copy()

# 解析対象：treated と BB9 が揃っている行
use = use.dropna(subset=["treated","BB9"])

# 極端な外れ値を軽くクリップ（任意）
if "BB9" in use.columns:
    use["BB9"] = use["BB9"].clip(lower=0, upper=10)
if "ERA" in use.columns:
    use["ERA"] = use["ERA"].clip(lower=0, upper=15)

In [ ]:
import statsmodels.formula.api as smf
import numpy as np

# 欠損のある行を除く
use = base_all.dropna(subset=["treated", "post"])

# 関数で共通化
def simple_did(dep):
    model = smf.ols(f"{dep} ~ treated * post", data=use.dropna(subset=[dep])).fit(cov_type="HC1")
    b3 = model.params.get("treated:post", np.nan)
    p3 = model.pvalues.get("treated:post", np.nan)
    print(f"\n=== DID for {dep} ===")
    print(f"β₃ (treated×post) = {b3:.4f}, p値 = {p3:.4f}")
    return model

# 実行
model_era = simple_did("ERA")
model_fip = simple_did("FIP")
model_bb9 = simple_did("BB9")


=== DID for ERA ===
β₃ (treated×post) = -0.0261, p値 = 0.9192

=== DID for FIP ===
β₃ (treated×post) = 0.2070, p値 = 0.2213

=== DID for BB9 ===
β₃ (treated×post) = 0.2133, p値 = 0.3225


In [ ]:
import statsmodels.formula.api as smf
import numpy as np

# 欠損除外
use = base_all.dropna(subset=["treated","post","FIP"])

# モデル①：Ageのみ追加
model_fip_age = smf.ols("FIP ~ treated * post + Age", data=use).fit(cov_type="HC1")

# 結果の抽出
b3 = model_fip_age.params.get("treated:post", np.nan)
p3 = model_fip_age.pvalues.get("treated:post", np.nan)
print(model_fip_age.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    FIP   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     18.63
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           1.35e-14
Time:                        14:58:04   Log-Likelihood:                -1128.2
No. Observations:                 737   AIC:                             2266.
Df Residuals:                     732   BIC:                             2289.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3.8263      0.349     10.968   

In [ ]:
import statsmodels.formula.api as smf
import numpy as np

# 共通の関数化
def run_did(dep_var):
    use = base_all.dropna(subset=["treated","post", dep_var])
    model = smf.ols(f"{dep_var} ~ treated * post + Age", data=use).fit(cov_type="HC1")
    b3 = model.params.get("treated:post", np.nan)
    p3 = model.pvalues.get("treated:post", np.nan)
    print(f"\n=== DID for {dep_var} (Age共変量) ===")
    print(model.summary())
    print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

# 3つの被説明変数で回す
for y in ["ERA", "FIP", "BB9"]:
    run_did(y)


=== DID for ERA (Age共変量) ===
                            OLS Regression Results                            
Dep. Variable:                    ERA   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     11.59
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           3.94e-09
Time:                        14:58:04   Log-Likelihood:                -1428.0
No. Observations:                 737   AIC:                             2866.
Df Residuals:                     732   BIC:                             2889.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3

In [ ]:
# モデル②：Age + IP を追加
model_fip_age_ip = smf.ols("FIP ~ treated * post + Age + IP", data=use).fit(cov_type="HC1")

b3 = model_fip_age_ip.params.get("treated:post", np.nan)
p3 = model_fip_age_ip.pvalues.get("treated:post", np.nan)
print(model_fip_age_ip.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    FIP   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     18.07
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           6.30e-17
Time:                        14:58:04   Log-Likelihood:                -1107.3
No. Observations:                 737   AIC:                             2227.
Df Residuals:                     731   BIC:                             2254.
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.3633      0.342     12.771   

In [ ]:
import statsmodels.formula.api as smf
import numpy as np

def run_did_with_age_ip(dep_var):
    use = base_all.dropna(subset=["treated", "post", dep_var, "Age", "IP"])
    model = smf.ols(f"{dep_var} ~ treated * post + Age + IP", data=use).fit(cov_type="HC1")
    b3 = model.params.get("treated:post", np.nan)
    p3 = model.pvalues.get("treated:post", np.nan)
    print(f"\n=== DID for {dep_var} (Age + IP共変量) ===")
    print(model.summary())
    print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

for y in ["ERA", "FIP", "BB9"]:
    run_did_with_age_ip(y)


=== DID for ERA (Age + IP共変量) ===
                            OLS Regression Results                            
Dep. Variable:                    ERA   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     10.85
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           4.34e-10
Time:                        14:58:04   Log-Likelihood:                -1408.2
No. Observations:                 737   AIC:                             2828.
Df Residuals:                     731   BIC:                             2856.
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    

In [ ]:
# モデル③：Age + IP + avg_velo_pre を追加
model_fip_age_ip_velo = smf.ols(
    "FIP ~ treated * post + Age + IP + avg_velo_pre",
    data=use.dropna(subset=["avg_velo_pre"])
).fit(cov_type="HC1")

b3 = model_fip_age_ip_velo.params.get("treated:post", np.nan)
p3 = model_fip_age_ip_velo.pvalues.get("treated:post", np.nan)
print(model_fip_age_ip_velo.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    FIP   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.119
Method:                 Least Squares   F-statistic:                     13.80
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           1.58e-10
Time:                        14:58:04   Log-Likelihood:                -488.60
No. Observations:                 390   AIC:                             987.2
Df Residuals:                     385   BIC:                             1007.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       12.9361      1.715      7.542   

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 4
  warnings.warn('covariance of constraints does not have full '


In [ ]:
import statsmodels.formula.api as smf
import numpy as np

def run_did_with_age_ip_velo(dep_var):
    use = base_all.dropna(subset=["treated", "post", dep_var, "Age", "IP", "avg_velo_pre"])
    model = smf.ols(f"{dep_var} ~ treated * post + Age + IP + avg_velo_pre", data=use).fit(cov_type="HC1")
    b3 = model.params.get("treated:post", np.nan)
    p3 = model.pvalues.get("treated:post", np.nan)
    print(f"\n=== DID for {dep_var} (Age + IP + avg_velo_pre共変量) ===")
    print(model.summary())
    print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

for y in ["ERA", "FIP", "BB9"]:
    run_did_with_age_ip_velo(y)


=== DID for ERA (Age + IP + avg_velo_pre共変量) ===
                            OLS Regression Results                            
Dep. Variable:                    ERA   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     6.485
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           4.66e-05
Time:                        14:58:04   Log-Likelihood:                -609.56
No. Observations:                 390   AIC:                             1229.
Df Residuals:                     385   BIC:                             1249.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 4
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 4
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 4
  warnings.warn('covariance of constraints does not have full '


In [ ]:
print("サンプル数:", len(use.dropna(subset=["FIP","avg_velo_pre","Age","IP","treated","post"])))

サンプル数: 390


In [ ]:
use_corr = use[["FIP","Age","IP","avg_velo_pre"]].dropna().corr()
print(use_corr)

                   FIP       Age        IP  avg_velo_pre
FIP           1.000000 -0.042546 -0.052246     -0.285927
Age          -0.042546  1.000000  0.009262     -0.218312
IP           -0.052246  0.009262  1.000000     -0.144722
avg_velo_pre -0.285927 -0.218312 -0.144722      1.000000


In [ ]:
model_fip_velo_only = smf.ols(
    "FIP ~ treated * post + avg_velo_pre",
    data=use.dropna(subset=["avg_velo_pre"])
).fit(cov_type="HC1")

b3 = model_fip_velo_only.params.get("treated:post", np.nan)
p3 = model_fip_velo_only.pvalues.get("treated:post", np.nan)
print(model_fip_velo_only.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    FIP   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     22.29
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           6.87e-10
Time:                        14:58:04   Log-Likelihood:                -494.31
No. Observations:                 390   AIC:                             994.6
Df Residuals:                     387   BIC:                             1007.
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       11.0107      1.514      7.271   

/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 2
  warnings.warn('covariance of constraints does not have full '


In [ ]:
# treated×post のクロス表
tab = use.dropna(subset=["avg_velo_pre"])[["treated","post"]].value_counts().unstack(fill_value=0)
print(tab)

# サンプル数
print("サンプル数（球速あり）:", len(use.dropna(subset=["avg_velo_pre","treated","post"])))


post       0
treated     
0.0      195
1.0      195
サンプル数（球速あり）: 390


In [ ]:
# 2022年: avg_velo_pre → avg_velo
merged_2022["avg_velo"] = merged_2022["avg_velo_pre"]

# 2023年: avg_velo_post → avg_velo
merged_2023["avg_velo"] = merged_2023["avg_velo_post"]

# 結合
base_all2 = pd.concat([merged_2022, merged_2023], ignore_index=True)
base_all2["post"] = (base_all2["year"] == 2023).astype(int)
base_all2["treated"] = base_all2["name_norm"].map(
    merged_2022.set_index("name_norm")["treated"]
)

# 欠損確認
print(base_all2[["year","avg_velo"]].isna().mean())
print(base_all2.groupby(["year","treated"]).size())

year        0.000000
avg_velo    0.004624
dtype: float64
year  treated
2022  0.0        195
      1.0        195
2023  0.0        172
      1.0        175
dtype: int64


In [ ]:
import statsmodels.formula.api as smf
import numpy as np

# 欠損除外
use = base_all2.dropna(subset=["treated","post","FIP","Age","IP","avg_velo"])

# モデル：Age + IP + avg_velo を共変量に
model_fip_velo = smf.ols(
    "FIP ~ treated * post + Age + IP + avg_velo",
    data=use
).fit(cov_type="HC1")

# DID効果抽出
b3 = model_fip_velo.params.get("treated:post", np.nan)
p3 = model_fip_velo.pvalues.get("treated:post", np.nan)

print(model_fip_velo.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    FIP   R-squared:                       0.203
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     22.53
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           1.96e-24
Time:                        14:58:04   Log-Likelihood:                -1077.9
No. Observations:                 737   AIC:                             2170.
Df Residuals:                     730   BIC:                             2202.
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       14.8939      1.632      9.129   

In [ ]:
import statsmodels.formula.api as smf
import numpy as np

for dep_var in ["ERA", "FIP", "BB9"]:
    use = base_all2.dropna(subset=["treated", "post", dep_var, "Age", "IP", "avg_velo"])
    model = smf.ols(
        f"{dep_var} ~ treated * post + Age + IP + avg_velo",
        data=use
    ).fit(cov_type="HC1")

    b3 = model.params.get("treated:post", np.nan)
    p3 = model.pvalues.get("treated:post", np.nan)

    print(f"\n=== DID for {dep_var} (Age + IP + avg_velo 共変量) ===")
    print(model.summary())
    print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")


=== DID for ERA (Age + IP + avg_velo 共変量) ===
                            OLS Regression Results                            
Dep. Variable:                    ERA   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     9.874
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           1.73e-10
Time:                        14:58:05   Log-Likelihood:                -1388.5
No. Observations:                 737   AIC:                             2791.
Df Residuals:                     730   BIC:                             2823.
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
I

In [ ]:
model_fip_spin = smf.ols(
    "FIP ~ treated * post + Age + IP + avg_velo + spin_rate_pre",
    data=use.dropna(subset=["spin_rate_pre"])
).fit(cov_type="HC1")

b3 = model_fip_spin.params.get("treated:post", np.nan)
p3 = model_fip_spin.pvalues.get("treated:post", np.nan)
print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

DID効果（treated×post）: β₃ = -0.0000, p値 = 0.0000


In [ ]:
print(use["year"].value_counts())
print(use["spin_rate_pre"].isna().value_counts())
print(use.groupby("year")["spin_rate_pre"].apply(lambda x: x.notna().mean()))

year
2022    390
2023    347
Name: count, dtype: int64
spin_rate_pre
False    390
True     347
Name: count, dtype: int64
year
2022    1.0
2023    0.0
Name: spin_rate_pre, dtype: float64


In [ ]:
# --- spin_rate の統一 ---
merged_2022["spin_rate"] = merged_2022["spin_rate_pre"]
merged_2023["spin_rate"] = merged_2023["spin_rate_post"]

# 結合
base_all3 = pd.concat([merged_2022, merged_2023], ignore_index=True)

# post / treated
base_all3["post"] = (base_all3["year"] == 2023).astype(int)
base_all3["treated"] = base_all3["name_norm"].map(
    merged_2022.set_index("name_norm")["treated"]
)

# 欠損除外してモデル作成
use3 = base_all3.dropna(subset=["treated","post","FIP","Age","IP","avg_velo","spin_rate"])

model_fip_spin = smf.ols(
    "FIP ~ treated * post + Age + IP + avg_velo + spin_rate",
    data=use3
).fit(cov_type="HC1")

b3 = model_fip_spin.params.get("treated:post", np.nan)
p3 = model_fip_spin.pvalues.get("treated:post", np.nan)
print(model_fip_spin.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    FIP   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     19.99
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           1.32e-24
Time:                        14:58:05   Log-Likelihood:                -1072.7
No. Observations:                 737   AIC:                             2161.
Df Residuals:                     729   BIC:                             2198.
Df Model:                           7                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       16.1895      1.669      9.703   

| モデル構成                   |  **共変量（control variables）**  | β₃（treated×post） |     p値     | 傾向              |
| :---------------------- | :--------------------------: | :--------------: | :--------: | :-------------- |
| **① ベースモデル**            |              なし              |      +0.2070     |    0.221   | 有意でないが正方向（悪化傾向） |
| **② + 年齢 (Age)**        |              Age             |      +0.2075     |    0.220   | 変化ほぼなし → 年齢影響小  |
| **③ + 投球回 (IP)**        |            Age, IP           |      +0.2091     |    0.203   | p値やや減少、傾向安定     |
| **④ + 球速 (avg_velo)**   |       Age, IP, avg_velo      |      +0.2060     |   0.1918   | 傾向変わらず、やや精度向上   |
| **⑤ + 回転数 (spin_rate)** | Age, IP, avg_velo, spin_rate |    **+0.2052**   | **0.1907** | 一貫した結果、最終モデル    |

In [ ]:
import statsmodels.formula.api as smf
import numpy as np

# --- データの準備 ---
merged_2022["spin_rate"] = merged_2022["spin_rate_pre"]
merged_2023["spin_rate"] = merged_2023["spin_rate_post"]

base_all3 = pd.concat([merged_2022, merged_2023], ignore_index=True)
base_all3["post"] = (base_all3["year"] == 2023).astype(int)
base_all3["treated"] = base_all3["name_norm"].map(
    merged_2022.set_index("name_norm")["treated"]
)

# --- モデル実行（ERA, FIP, BB9） ---
for dep_var in ["ERA", "FIP", "BB9"]:
    use3 = base_all3.dropna(subset=["treated", "post", dep_var, "Age", "IP", "avg_velo", "spin_rate"])

    model = smf.ols(
        f"{dep_var} ~ treated * post + Age + IP + avg_velo + spin_rate",
        data=use3
    ).fit(cov_type="HC1")

    b3 = model.params.get("treated:post", np.nan)
    p3 = model.pvalues.get("treated:post", np.nan)

    print(f"\n=== DID for {dep_var} (Age + IP + avg_velo + spin_rate 共変量) ===")
    print(model.summary())
    print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")


=== DID for ERA (Age + IP + avg_velo + spin_rate 共変量) ===
                            OLS Regression Results                            
Dep. Variable:                    ERA   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     11.65
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           4.29e-14
Time:                        14:58:05   Log-Likelihood:                -1386.0
No. Observations:                 737   AIC:                             2788.
Df Residuals:                     729   BIC:                             2825.
Df Model:                           7                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [ ]:
import statsmodels.formula.api as smf
import numpy as np

# 欠損除外
use = base_all3.dropna(subset=["treated","post","BB9"])

# モデル①：共変量なし
model_bb9_0 = smf.ols("BB9 ~ treated * post", data=use).fit(cov_type="HC1")

b3 = model_bb9_0.params.get("treated:post", np.nan)
p3 = model_bb9_0.pvalues.get("treated:post", np.nan)
print(model_bb9_0.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    BB9   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     10.19
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           1.41e-06
Time:                        14:58:05   Log-Likelihood:                -1304.9
No. Observations:                 737   AIC:                             2618.
Df Residuals:                     733   BIC:                             2636.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3.0154      0.076     39.746   

In [ ]:
# モデル②：Ageを共変量に追加
model_bb9_age = smf.ols("BB9 ~ treated * post + Age", data=use).fit(cov_type="HC1")

b3 = model_bb9_age.params.get("treated:post", np.nan)
p3 = model_bb9_age.pvalues.get("treated:post", np.nan)
print(model_bb9_age.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    BB9   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     9.421
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           1.99e-07
Time:                        14:58:05   Log-Likelihood:                -1303.3
No. Observations:                 737   AIC:                             2617.
Df Residuals:                     732   BIC:                             2640.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3.7701      0.417      9.052   

In [ ]:
# モデル③：Age + IP
model_bb9_age_ip = smf.ols("BB9 ~ treated * post + Age + IP", data=use).fit(cov_type="HC1")

b3 = model_bb9_age_ip.params.get("treated:post", np.nan)
p3 = model_bb9_age_ip.pvalues.get("treated:post", np.nan)
print(model_bb9_age_ip.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    BB9   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     25.06
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           2.32e-23
Time:                        14:58:05   Log-Likelihood:                -1255.3
No. Observations:                 737   AIC:                             2523.
Df Residuals:                     731   BIC:                             2550.
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.7823      0.395     12.097   

In [ ]:
# モデル④：Age + IP + avg_velo
model_bb9_velo = smf.ols(
    "BB9 ~ treated * post + Age + IP + avg_velo",
    data=use.dropna(subset=["avg_velo"])
).fit(cov_type="HC1")

b3 = model_bb9_velo.params.get("treated:post", np.nan)
p3 = model_bb9_velo.pvalues.get("treated:post", np.nan)
print(model_bb9_velo.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    BB9   R-squared:                       0.163
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     21.37
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           3.46e-23
Time:                        14:58:05   Log-Likelihood:                -1254.9
No. Observations:                 737   AIC:                             2524.
Df Residuals:                     730   BIC:                             2556.
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3.1530      1.744      1.808   

In [ ]:
# モデル⑤：Age + IP + avg_velo + spin_rate
model_bb9_spin = smf.ols(
    "BB9 ~ treated * post + Age + IP + avg_velo + spin_rate",
    data=use.dropna(subset=["avg_velo","spin_rate"])
).fit(cov_type="HC1")

b3 = model_bb9_spin.params.get("treated:post", np.nan)
p3 = model_bb9_spin.pvalues.get("treated:post", np.nan)
print(model_bb9_spin.summary())
print(f"\nDID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

                            OLS Regression Results                            
Dep. Variable:                    BB9   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     18.96
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           2.47e-23
Time:                        14:58:05   Log-Likelihood:                -1253.0
No. Observations:                 737   AIC:                             2522.
Df Residuals:                     729   BIC:                             2559.
Df Model:                           7                                         
Covariance Type:                  HC1                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.1595      1.834      1.178   

| モデル         |  **共変量（control variables）**  | β₃（treated×post） |     p値     | 傾向             |
| :---------- | :--------------------------: | :--------------: | :--------: | :------------- |
| ① ベースモデル    |              なし              |      +0.2133     |   0.3225   | テンポ遅い投手で与四球やや増 |
| ② + 年齢      |              Age             |      +0.2115     |   0.3261   | 年齢の影響は小さい      |
| ③ + 投球回     |            Age, IP           |      +0.2147     |   0.2881   | わずかにp値低下（精度改善） |
| ④ + 球速      |       Age, IP, avg_velo      |      +0.2152     |   0.2871   | 傾向・方向ともに安定     |
| **⑤ + 回転数** | Age, IP, avg_velo, spin_rate |    **+0.2157**   | **0.2848** | 安定した正の効果。ロバスト。 |


In [ ]:
# 先発ダミー（IP >= 80回を starter=1、それ以外=0）
use["starter_dummy"] = (use["IP"] >= 80).astype(int)

print(use["starter_dummy"].value_counts())

starter_dummy
0    456
1    281
Name: count, dtype: int64


/tmp/ipython-input-2870871812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  use["starter_dummy"] = (use["IP"] >= 80).astype(int)


In [ ]:
# モデル: FIP × 投手タイプ
model_fip_type = smf.ols(
    "FIP ~ treated * post + Age + IP + avg_velo + spin_rate + starter_dummy",
    data=use.dropna(subset=["avg_velo","spin_rate"])
).fit(cov_type="HC1")

b3 = model_fip_type.params.get("treated:post", np.nan)
p3 = model_fip_type.pvalues.get("treated:post", np.nan)
b_type = model_fip_type.params.get("starter_dummy", np.nan)
p_type = model_fip_type.pvalues.get("starter_dummy", np.nan)

print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")
print(f"先発ダミー（starter_dummy）: β = {b_type:.4f}, p値 = {p_type:.4f}")

DID効果（treated×post）: β₃ = 0.1964, p値 = 0.1979
先発ダミー（starter_dummy）: β = 0.9947, p値 = 0.0000


In [ ]:
# 先発投手のみを抽出
use_starter = use.query("starter_dummy == 1").copy()
print("先発投手サンプル数:", use_starter.shape[0])

# FIPを目的変数としたDID
model_fip_starter = smf.ols(
    "FIP ~ treated * post + Age + IP + avg_velo + spin_rate",
    data=use_starter.dropna(subset=["avg_velo","spin_rate"])
).fit(cov_type="HC1")

# DID効果（treated×post）
b3 = model_fip_starter.params.get("treated:post", np.nan)
p3 = model_fip_starter.pvalues.get("treated:post", np.nan)

print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

先発投手サンプル数: 281
DID効果（treated×post）: β₃ = 0.2722, p値 = 0.1589


In [ ]:
# 各年ごとの投手名（正規化済み）集合を作る
players_2022 = set(use.query("year == 2022 and starter_dummy == 1")["name_norm"])
players_2023 = set(use.query("year == 2023 and starter_dummy == 1")["name_norm"])

# 共通している投手数
common_pitchers = players_2022 & players_2023
print("共通している先発投手数:", len(common_pitchers))

共通している先発投手数: 98


In [ ]:
# ① 2022・2023両方に出ている先発投手を特定
players_2022 = set(use.query("year == 2022 and starter_dummy == 1")["name_norm"])
players_2023 = set(use.query("year == 2023 and starter_dummy == 1")["name_norm"])
common_pitchers = players_2022 & players_2023

print("共通している先発投手数:", len(common_pitchers))

# ② 共通投手のみを抽出
use_starter_common = use.query("starter_dummy == 1 and name_norm in @common_pitchers").copy()
print("共通先発観測数:", use_starter_common.shape[0])

# ③ DID回帰（FIP）
model_fip_common = smf.ols(
    "FIP ~ treated * post + Age + IP + avg_velo + spin_rate",
    data=use_starter_common.dropna(subset=["avg_velo","spin_rate"])
).fit(cov_type="HC1")

# ④ 結果出力
b3 = model_fip_common.params.get("treated:post", np.nan)
p3 = model_fip_common.pvalues.get("treated:post", np.nan)

print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

共通している先発投手数: 98
共通先発観測数: 196
DID効果（treated×post）: β₃ = 0.2186, p値 = 0.3383


In [ ]:
# ① リリーフ投手のみ抽出
use_reliever = use.query("starter_dummy == 0").copy()
print("リリーフ投手サンプル数:", use_reliever.shape[0])

# ② DID回帰（FIPを目的変数）
model_fip_reliever = smf.ols(
    "FIP ~ treated * post + Age + IP + avg_velo + spin_rate",
    data=use_reliever.dropna(subset=["avg_velo","spin_rate"])
).fit(cov_type="HC1")

# ③ 結果の抽出
b3 = model_fip_reliever.params.get("treated:post", np.nan)
p3 = model_fip_reliever.pvalues.get("treated:post", np.nan)

print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

リリーフ投手サンプル数: 456
DID効果（treated×post）: β₃ = 0.2658, p値 = 0.2261


| 分析対象           | サンプル数 | DID効果 β₃ (treated×post) |    p値    | 解釈                |
| :------------- | :---: | :---------------------: | :------: | :---------------- |
| **全体（共変量なし）**  |  約730 |          +0.21          |   0.22   | 軽度の悪化傾向（有意でない）    |
| **全体（共変量あり）**  |  約730 |          +0.20          |   0.19   | 安定した方向性（悪化）       |
| **先発限定**       |  281  |        **+0.27**        | **0.16** | 悪化効果がやや強まる（最も明確）  |
| **共通先発（2年連続）** |  196  |          +0.22          |   0.33   | 同一投手でも悪化傾向維持      |
| **リリーフ限定**     |  456  |          +0.27          |   0.23   | 方向性は同じ（悪化）だが有意でない |

In [ ]:
# 先発のみ
use_starter = use.query("starter_dummy == 1").copy()

model_bb9_starter = smf.ols(
    "BB9 ~ treated * post + Age + IP + avg_velo + spin_rate",
    data=use_starter.dropna(subset=["avg_velo","spin_rate"])
).fit(cov_type="HC1")

b3 = model_bb9_starter.params.get("treated:post", np.nan)
p3 = model_bb9_starter.pvalues.get("treated:post", np.nan)

print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

DID効果（treated×post）: β₃ = 0.3530, p値 = 0.1217


In [ ]:
# リリーフのみ
use_reliever = use.query("starter_dummy == 0").copy()

model_bb9_reliever = smf.ols(
    "BB9 ~ treated * post + Age + IP + avg_velo + spin_rate",
    data=use_reliever.dropna(subset=["avg_velo","spin_rate"])
).fit(cov_type="HC1")

b3 = model_bb9_reliever.params.get("treated:post", np.nan)
p3 = model_bb9_reliever.pvalues.get("treated:post", np.nan)

print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")

DID効果（treated×post）: β₃ = 0.2778, p値 = 0.3579


| 投手タイプ      | サンプル数 | β₃ (treated×post) |     p値     | 解釈                            |
| :--------- | :---: | :---------------: | :--------: | :---------------------------- |
| **先発投手**   |  約281 |    **+0.3530**    | **0.1217** | 四球率が顕著に悪化傾向（10%有意水準に近い）       |
| **リリーフ投手** |  約456 |      +0.2778      |   0.3579   | 四球率がやや上昇するが、ばらつきが大きく統計的に有意でない |

In [ ]:
import numpy as np

# G, GS がない or 0 の行は先に除外しておく
use_gs = use.dropna(subset=["G", "GS"]).copy()
use_gs = use_gs[use_gs["G"] > 0].copy()

# 先発割合（先発登板数 / 登板試合数）
use_gs["start_ratio"] = use_gs["GS"] / use_gs["G"]

# 先発ダミー（登板の半分以上が先発なら starter=1）
use_gs["starter_dummy"] = (use_gs["start_ratio"] >= 0.5).astype(int)

print(use_gs["starter_dummy"].value_counts())

# 先発投手のみを抽出
use_starter = use_gs.query("starter_dummy == 1").copy()
print("先発投手サンプル数:", use_starter.shape[0])

# FIPを目的変数としたDID（共変量はそのまま）
model_fip_starter = smf.ols(
    "FIP ~ treated * post + Age + IP + avg_velo + spin_rate",
    data=use_starter.dropna(subset=["avg_velo", "spin_rate"])
).fit(cov_type="HC1")

b3 = model_fip_starter.params.get("treated:post", np.nan)
p3 = model_fip_starter.pvalues.get("treated:post", np.nan)

print(f"DID効果（treated×post）: β₃ = {b3:.4f}, p値 = {p3:.4f}")


starter_dummy
0    401
1    336
Name: count, dtype: int64
先発投手サンプル数: 336
DID効果（treated×post）: β₃ = 0.1726, p値 = 0.4407


In [ ]:
# 本編 DID で使っているデータ（base_all2 とか）から、投手集合を取る
analysis = base_all2.dropna(subset=["treated","FIP"]).copy()
pitchers = analysis["name_norm"].unique()

# placebo用データを「本編と同じ投手だけ」に絞る
did_placebo_sub = did_placebo[did_placebo["name_norm"].isin(pitchers)].copy()

print(pd.crosstab(did_placebo_sub["treated"], did_placebo_sub["fake_post"]))

def placebo_sub(outcome):
    m = smf.ols(f"{outcome} ~ treated * fake_post", data=did_placebo_sub).fit(cov_type="HC1")
    print(f"\n[Placebo DID-sub] {outcome}: β3 = {m.params.get('treated:fake_post', np.nan):.4f}, p値 = {m.pvalues.get('treated:fake_post', np.nan):.4f}")
    return m

placebo_sub("ERA"); placebo_sub("FIP"); placebo_sub("BB9")

fake_post    0    1
treated            
0.0        200  231
1.0        236  245

[Placebo DID-sub] ERA: β3 = -0.5061, p値 = 0.2490

[Placebo DID-sub] FIP: β3 = -0.2742, p値 = 0.2889

[Placebo DID-sub] BB9: β3 = -0.4500, p値 = 0.0681
